In [1]:
import pandas as pd
import numpy as np
import arxivscraper
from random import randint
import matplotlib.pyplot as plt
import datetime
import json

#### Make authors, papers and authors_paper without rules

WARNING: Running this cell only if you want to extract all the data again, it takes too long

In [2]:
# Getting data computer science session
if False:
    scraper_cs = arxivscraper.Scraper(category='math', date_from='2007-01-01',date_until='2009-12-30')
    output = scraper_cs.scrape()

    with open('../datasets/_cs_scraper.json', 'w') as fp:
        json.dump(output, fp)

http://export.arxiv.org/oai2?verb=ListRecords&from=2007-01-01&until=2009-12-30&metadataPrefix=arXiv&set=math
fetching up to  1000 records...
fetching up to  2000 records...
fetching up to  3000 records...
Got 503. Retrying after 15 seconds.
fetching up to  3000 records...
fetching up to  4000 records...
Got 503. Retrying after 15 seconds.
fetching up to  4000 records...
fetching up to  5000 records...
Got 503. Retrying after 15 seconds.
fetching up to  5000 records...
fetching up to  6000 records...
Got 503. Retrying after 15 seconds.
fetching up to  6000 records...
fetching up to  7000 records...
Got 503. Retrying after 15 seconds.
fetching up to  7000 records...
fetching up to  8000 records...
Got 503. Retrying after 15 seconds.
fetching up to  8000 records...
fetching up to  9000 records...
Got 503. Retrying after 15 seconds.
fetching up to  9000 records...
fetching up to  10000 records...
Got 503. Retrying after 15 seconds.
fetching up to  10000 records...
fetching up to  11000 rec

In [3]:
if True:
    
    with open('../datasets/_cs_scraper.json') as fp:
        output = json.loads(fp.read())
        
    #df = pd.DataFrame.from_dict(output)
    # Getting data stat session
#     scraper_stat = arxivscraper.Scraper(category='physics:nucl-th', date_from='2007-01-01',date_until='2017-12-30')
#     outputs = scraper_stat.scrape()
#     df2 = pd.DataFrame.from_dict(outputs)
    
    # generate authors
    authors = []

#     output = output[0:len(outputs)]
    for aut in output:
        for i, a in enumerate(aut['authors']):
            authors.append({'idauthor': f'auth_{(i + randint(1,21)*5)}_{(i + randint(1,21))}', 'name': a})

#     for aut in outputs:
#         for i, a in enumerate(aut['authors']):
#             authors.append({'idauthor': f'aoz{(i + randint(1,21)*5)}xyz{(i + randint(1,21))}', 'name': a})

    authors
    
    #Generate papers
    papers = []

    for aut in output:
        papers.append({'idpaper': aut['id'], 'title': aut['title'], 'abstract': aut['abstract'], 'date_create': aut['created'], 'area': aut['categories'], 'authors': aut['authors']})

#     for aut in outputs:
#         papers.append({'idpaper': aut['id'], 'title': aut['title'], 'abstract': aut['abstract'], 'date_create': aut['created'], 'area': aut['categories'], 'authors': aut['authors']})

    papers
    
    # Generate authors_paper
    author_paper = []

    for p in papers:
        for a in authors:
            if a['name'] in p['authors']:
                author_paper.append({'author_idauthor': a['idauthor'], 'paper_idpaper': p['idpaper'], 'date_create': p['date_create']})
    author_paper
    
    # Save .json files
    import json
    with open('../datasets/_authors.json', 'w') as fp:
        json.dump(authors, fp)

    with open('../datasets/_papers.json', 'w') as fp:
        json.dump(papers, fp)

    with open('../datasets/_author_paper.json', 'w') as fp:
        json.dump(author_paper, fp)

#### Download dataset

#### ** Step initial **
WARNING: This cell should only be executed if it does not have the data on the local machine

In [4]:
if False:
    !wget https://www.dropbox.com/s/s5mysx0b6sp7jvw/authors.json -P ../datasets/
    !wget https://www.dropbox.com/s/ktvn3q2ijfka9zn/authors_paper.json -P ../datasets/
    !wget https://www.dropbox.com/s/jbqvl5htdv41c33/papers.json -P ../datasets/

#### Apply rules

1. Remove papers with only one author
2. All areas that are in year 1 must be in year 2
3. Remove authors who appear in year 1 and do not appear in year 2
4. Remove authors who are only in year 2

In [5]:
if True:
    # load dicts
    import json
    with open('../datasets/_authors.json') as fp:
        authors = json.loads(fp.read())

    with open('../datasets/_papers.json') as fp:
        papers = json.loads(fp.read())

    #     with open('../datasets/_author_paper.json') as fp:
    #         author_paper = json.loads(fp.read())

    # drop single authors
    papers_with_multiple_authors = []

    for p in papers:
        if len(p['authors']) > 1:
            papers_with_multiple_authors.append(p)

    papers = papers_with_multiple_authors

    # Generate authors_paper
    author_paper = []

    for p in papers:
        for a in authors:
            if a['name'] in p['authors']:
                author_paper.append({'author_idauthor': a['idauthor'], 'paper_idpaper': p['idpaper'], 'date_create': p['date_create']})
    len(author_paper)

    # Save .json files
    import json
    with open('../datasets/_authors.json', 'w') as fp:
        json.dump(authors, fp)

    with open('../datasets/_papers.json', 'w') as fp:
        json.dump(papers, fp)

    with open('../datasets/_author_paper.json', 'w') as fp:
        json.dump(author_paper, fp)

In [6]:
if True:
    from dateutil.relativedelta import relativedelta
    import datetime
    
    # Creates two auxiliary lists with each year's publications
    year_one = []
    year_two = []
    year_limit = datetime.datetime.strptime("2008-01-01", "%Y-%m-%d").timestamp() * 1000

#     for ap in author_paper:
#         dt_c = datetime.datetime.strptime(ap['date_create'], "%Y-%m-%d").timestamp() * 1000
#         if dt_c > year_limit:
#             year_two.append(ap)
#         else:
#             year_one.append(ap)

#     print(len(author_paper), len(year_two), len(year_one))

    def verify_author_at_year(id_author, list_year):
        for i in list_year:
            if id_author in i['author_idauthor']:
                return True
        return False

#     author_paper_on_two_year = []
#     single_ids_author = []
#     for ap in autor_paper:
#         if ap['author_idauthor'] not in single_ids_author:
#             single_ids_author.append(ap['author_idauthor'])
#             exists_yer_one = verify_author_at_year(ap['author_idauthor'], year_one)
#             exists_yer_two = verify_author_at_year(ap['author_idauthor'], year_two)
#             if exists_yer_one and exists_yer_two:
#                 author_paper_on_two_year.append(ap)

#     print(len(author_paper_on_two_year))

    # removes non-published authors within two years

    filter_authors = []

#     for a in authors:
#         for apty in author_paper_on_two_year:
#             if a['idauthor'] in apty['author_idauthor']:
#                 filter_authors.append(a)
#                 continue

#     print(len(authors), len(filter_authors))


    # removes unused papers 

#     filter_papers = []

#     for p in papers:
#         for apty2 in author_paper_on_two_year:
#             if p['idpaper'] in apty2['paper_idpaper']:
#                 filter_papers.append(p)

#     print(len(papers), len(filter_papers))

    # Save .json files
    import json
#     with open('../datasets/_authors.json', 'w') as fp:
#         json.dump(filter_authors, fp)

#     with open('../datasets/_papers.json', 'w') as fp:
#         json.dump(filter_papers, fp)

#     with open('../datasets/_author_paper.json', 'w') as fp:
#         # json.dump(author_paper_on_two_year, fp)
#         json.dump(author_paper, fp)

#### Load dataframes authors

In [7]:
df_authors = pd.read_json('../datasets/_authors.json')
df_authors.drop_duplicates(subset=['idauthor'], keep=False, inplace=True)
df_authors.head()

,idauthor,name
1131,auth_106_26,lasenby
1132,auth_32_27,martinez-gonzalez
1229,auth_84_23,poor
1626,auth_69_6,lotker
3258,auth_105_22,labourie


#### Load dataframes author_paper

In [40]:
df_author_paper = pd.read_json('../datasets/_author_paper.json')
#df_author_paper.drop_duplicates(subset=['date_create', 'paper_idpaper'], keep=False, inplace=True)
df_filter = df_author_paper[df_author_paper['date_create'] > '2004-01-01']
df_filter = df_filter[df_filter['date_create'] < '2007-01-01']

In [42]:
df_filter.head()

,author_idauthor,date_create,paper_idpaper
1716429,auth_85_1,2004-02-12,astro-ph/0402324
1716430,auth_106_12,2004-02-12,astro-ph/0402324
1716431,auth_100_13,2004-02-12,astro-ph/0402324
1716432,auth_55_8,2004-02-12,astro-ph/0402324
1716433,auth_81_17,2004-02-12,astro-ph/0402324


In [44]:
auth_ppr = df_filter.to_dict()

# with open('../datasets/_author_paper_filter.json', 'w') as fp:
#         json.dump(auth_ppr, fp)

In [45]:
auth_ppr

{'author_idauthor': {1716429: 'auth_85_1',
  1716430: 'auth_106_12',
  1716431: 'auth_100_13',
  1716432: 'auth_55_8',
  1716433: 'auth_81_17',
  1716434: 'auth_45_2',
  1716435: 'auth_56_5',
  1716436: 'auth_75_3',
  1716437: 'auth_41_10',
  1716438: 'auth_35_10',
  1716439: 'auth_95_19',
  1716440: 'auth_40_18',
  1716441: 'auth_20_14',
  1716442: 'auth_23_10',
  1716443: 'auth_105_9',
  1716444: 'auth_77_3',
  1716445: 'auth_55_2',
  1716446: 'auth_6_8',
  1716447: 'auth_70_14',
  1716448: 'auth_101_17',
  1716449: 'auth_18_11',
  1716450: 'auth_66_5',
  1716451: 'auth_85_8',
  1716452: 'auth_11_4',
  1716453: 'auth_26_20',
  1716454: 'auth_26_9',
  1716455: 'auth_7_21',
  1716456: 'auth_60_10',
  1716457: 'auth_30_8',
  1716458: 'auth_93_10',
  1716459: 'auth_6_14',
  1716460: 'auth_10_5',
  1716461: 'auth_85_9',
  1716462: 'auth_92_17',
  1716463: 'auth_37_4',
  1716464: 'auth_105_2',
  1716465: 'auth_7_8',
  1716466: 'auth_67_15',
  1716467: 'auth_12_18',
  1716468: 'auth_18_6',


In [26]:
df_author_paper.head()

,author_idauthor,date_create,paper_idpaper
0,auth_35_17,2007-03-30,0704.0002
1,auth_41_7,2007-03-30,0704.0002
2,auth_100_17,2007-03-30,0704.0002
3,auth_61_5,2007-03-30,0704.0002
4,auth_56_10,2007-03-30,0704.0002


In [29]:
len(df_author_paper)

4250129

#### Load dataframes paper

In [9]:
df_paper = pd.read_json('../datasets/_papers.json')
df_paper.drop(columns=['authors'], axis=1, inplace=True)
df_paper.drop_duplicates(subset=['area', 'idpaper', 'title'], keep=False, inplace=True)
df_paper.tail()

,abstract,area,date_create,idpaper,title
43813,a n=4 supersymmetric matrix kp hierarchy is pr...,solv-int hep-th math-ph math.mp nlin.si,1999-06-29,solv-int/9907004,n=2 local and n=4 nonlocal reductions of super...
43814,we introduce a class of cellular automata asso...,solv-int math.qa nlin.si,1999-07-27,solv-int/9907020,soliton cellular automata associated with crys...
43815,for the integrable case of the discrete self-t...,solv-int hep-th math.ca math.qa nlin.si,1999-07-31,solv-int/9908002,quantum backlund transformation for the integr...
43816,a family of completely integrable nonlinear de...,solv-int math.qa nlin.si,1999-11-03,solv-int/9911004,integrable deformations of oscillator chains f...
43817,three inequivalent real forms of the complex n...,solv-int hep-th math-ph math.mp nlin.si,1999-11-04,solv-int/9911005,a note on real forms of the complex n=4 supers...


#### Extract features with LSA from abstract 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.externals import joblib

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=50000, stop_words="english", use_idf=True, smooth_idf=True)
X = tfidf_vectorizer.fit_transform(df_paper['abstract'])

models = '../models/'

with open(f'{models}_tf_idf_vectorize.pkl', 'wb') as mtfidf:
    joblib.dump(tfidf_vectorizer, mtfidf)

svd = TruncatedSVD(n_components=400, algorithm='randomized',
                   n_iter=10)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
lsa.fit_transform(X)

with open(f'{models}_lsa.pkl', 'wb') as fin:
    joblib.dump(lsa, fin)

tf_idf = joblib.load(f'{models}_tf_idf_vectorize.pkl')

tf_idf_features = tf_idf.transform(df_paper['abstract'])

lsa = joblib.load(f'{models}_lsa.pkl')

X = lsa.transform(tf_idf_features)

In [ ]:
X

#### Clustering K-MEANS Sklearn

In [ ]:
def kmeans(data):    
    from sklearn.cluster import KMeans
    %matplotlib inline
    import matplotlib.pyplot as plt
    import seaborn as sns; sns.set()

    model = KMeans(n_clusters=2, verbose=False)
    model.fit(data)

    y_kmeans = model.predict(data)
        
    plt.scatter(data[:, 0], data[:, 1], c=y_kmeans, s=50, cmap='viridis')

    centers = model.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

In [ ]:
kmeans(X)